In [28]:
instance = InstanceId = instanceId
ApiUrl = apiUrl
SchoolYear = schoolYear
DistrictId = DistrictID = districtID = districtId
apiLimit = batchLimit
prepareEdFiMetaData = prepareEdFiMetadata

### URL Initializations

In [29]:
%run OEA/modules/Ed-Fi/v0.7/src/utilities/edfi_v0_7_fetch_urls

In [30]:
instance_id = instanceId
school_year = schoolYear
api_url = apiUrl

edfi_api_manager = EdFiApiManager(api_url, instance_id, school_year)
edfi_api_manager.update_urls()
edfi_api_manager.set_other_metadata()

dependenciesUrl = edfi_api_manager.dependencies_url
openApiMetadataUrl = edfi_api_manager.openapi_metadata_url
dataManagementUrl = edfi_api_manager.data_management_url
authUrl = edfi_api_manager.auth_url

changeQueriesUrl = edfi_api_manager.get_referenced_url('Change-Queries')
changeQueriesUrl = changeQueriesUrl[:-13].replace('/metadata/', '/')
swagger_url = swaggerUrl = edfi_api_manager.get_referenced_url('Resources')

apiVersion = edfi_api_manager.api_version
apiVersion = apiVersion[1:] if apiVersion.startswith('v') else apiVersion

### OEA Initializations

In [31]:
%run OEA/modules/Ed-Fi/v0.7/src/utilities/edfi_v0_7_edfi_py

In [32]:
oea = EdFiOEAChild()   
oea.set_workspace(workspace)

### Error Logging Initializations

In [33]:
error_logger = ErrorLogging(spark = spark, 
                            oea = oea, 
                            logger = logger)

### Main Code

In [34]:
from datetime import datetime
import math
source_path = f'stage1/Transactional/Ed-Fi/{apiVersion}/DistrictId={districtId}/SchoolYear={schoolYear}/metadata-assets/frequency_etl.csv'  
destination_path = source_path #f'stage1/Transactional/Ed-Fi/{apiVersion}/DistrictId={districtId}/SchoolYear={schoolYear}/metadata-assets/frequency_based_etl.csv'  
logs_path = f"stage1/Transactional/Ed-Fi/{apiVersion}/DistrictId={districtId}/SchoolYear={schoolYear}/metadata-assets/_frequency_etl_logs/run_logs_{datetime.today().strftime('%Y-%m-%d')}.csv"

In [35]:
exception = None
entitiesToFetch = "All"
from datetime import datetime
import math

retry_strategy = Retry(total = 3,
                       backoff_factor = 1,
                       status_forcelist = [429, 500, 502, 503, 504],
                       allowed_methods = ["HEAD", "GET", "OPTIONS", "POST", "DELETE"])
try:
    edfiAPIClient = EdFiClient(workspace = workspace, 
                                    kvName = kvName, #NOTE: Default to None 
                                    moduleName = moduleName, 
                                    authUrl = authUrl, 
                                    dataManagementUrl = dataManagementUrl, 
                                    changeQueriesUrl = changeQueriesUrl, 
                                    dependenciesUrl = dependenciesUrl, 
                                    apiVersion = apiVersion, 
                                    batchLimit = batchLimit, 
                                    minChangeVer = minChangeVer, 
                                    maxChangeVer = maxChangeVer,
                                    landingDateTimeFormat = landingDateTimeFormat,
                                    schoolYear = schoolYear,
                                    districtId = districtId,
                                    kvSecret_clientId = client_id,
                                    kvSecret_clientSecret = client_secret_id,
                                    retry_strategy = retry_strategy,
                                    threadMode = True,
                                    devMode = kvSecret_devMode,
                                    error_logger = error_logger)
except Exception as e:
    exception = e
    logger.error(exception)

    raise(exception)

In [36]:
processor = EntityFrequencyProcessor(oea = oea, 
                                     filepath = source_path, 
                                     highFrequentDelta = highFrequentDelta,#0.005, 
                                     moderateFrequentDelta = moderateFrequentDelta, #5, 
                                     lowFrequentDelta = lowFrequentDelta, #10, 
                                     descriptorsDelta = descriptorsDelta) #360)

In [37]:
processor.load_lookup_df()
processor.write_lookup_df(logs_path)
entities_to_etl, _ = processor.return_entities_to_etl()
if len(entities_to_etl) != 0:
    edfiAPIClient.landEntities(entities_to_etl, 
                               debugMode = False, 
                               fetchHistory = fetchHistory,
                               thread_max_workers = 4,
                               requestedUrls=dict())
else:
    logger.info("No entities to be fetched has been specified")

In [38]:
try:
    edfiAPIClient.landEntity(resource = '/ed-fi/schoolYearTypes',
                          minChangeVersion = None,
                          maxChangeVersion = None, 
                          debugMode = True,
                          requestedUrls = list())
except Exception as schoolYearTypesError:
    logger.exception('[LANDING ENTITY] Landing of schoolYearTypes failed')
    logger.exception(schoolYearTypesError)
    raise(schoolYearTypesError)

### Writing Logs

In [39]:
if edfiAPIClient.error_logger.entity_logs != []:
    logger.info('[LANDING ERROR LOGGING] Writing Entity Level Error Logs')
    df = edfiAPIClient.error_logger.create_spark_df('entity')
    edfiAPIClient.error_logger.write_logs_to_delta_lake(df = df, 
                                                        log_type = 'entity',
                                                        destination_url = edfiAPIClient.error_logger.to_logs_url('etl-logs/log_type=entity'))
    edfiAPIClient.error_logger.add_etl_logs_to_lake_db(db_name = f'ldb_{workspace}_edfi_etl_logs',
                                                        logs_base_path = 'etl-logs',
                                                        log_type = 'entity',
                                                        overwrite = True)

In [ ]:
# %run Dev/Ed-Fi/v0.7/dev_v0_7_EdFi_Ingest

In [57]:
# %run Dev/Ed-Fi/v0.7/dev_v0_7_EdFi_Refine